# **Réalisez un traitement dans un environnement Big Data sur le Cloud**

## **Import des librairies**

In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

2024-01-29 13:05:02.560976: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 13:05:02.561026: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 13:05:02.561048: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 13:05:02.566115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## **Définition des PATH pour charger les images et enregistrer les résultats**

In [2]:
PATH = os.getcwd()
PATH_Data = PATH + '/data/Test1'
PATH_Result = PATH + '/data/Results'
print('PATH :        ' + PATH\
      + '\nPATH_Data :   ' + PATH_Data \
      + '\nPATH_Result : ' + PATH_Result)

PATH :        /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers
PATH_Data :   /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Test1
PATH_Result : /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Results


## **Création de la SparkSession**

In [3]:
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Nous créons également la variable "**sc**" qui est un **SparkContext** issue de la variable **spark** :

In [4]:
sc = spark.sparkContext

Affichage des informations de Spark en cours d'execution :

In [5]:
spark

## **Traitement des données**

### **Chargement des données**

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

Je ne conserve que le **path** de l'image et j'ajoute une colonne contenant les **labels** de chaque image :

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|path                                                                                                                                                                                                 |label             |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|file:/mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Clou

### **Préparation du modèle**

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** <br />
    issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec:
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [8]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

2024-01-29 13:05:14.374240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-29 13:05:14.393912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-29 13:05:14.393949: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-29 13:05:14.394987: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-29 13:05:14.395017: I tensorflow/compile

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

Affichage du résumé de notre nouveau modèle où nous constatons <br />
que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [10]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser <br />
ensuite les poids aux différents workeurs.

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

<u>Mettons cela sous forme de fonction</u> :

In [12]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

### **Définition du processus de chargement des images et application <br/>de leur featurisation à travers l'utilisation de pandas UDF**

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.

<u>L'empilement des appels est la suivante</u> :

- Pandas UDF
  - featuriser une série d'images pd.Series
   - prétraiter une image

In [16]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

### **Exécution des actions d'extraction de features**

Les Pandas UDF, sur de grands enregistrements (par exemple, de très grandes images), <br />
peuvent rencontrer des erreurs de type Out Of Memory (OOM).<br />
Si vous rencontrez de telles erreurs dans la cellule ci-dessous, <br />
essayez de réduire la taille du lot Arrow via 'maxRecordsPerBatch'

Je n'utiliserai pas cette commande dans ce projet <br />
et je laisse donc la commande en commentaire.

In [ ]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [17]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

In [20]:
print(f"Les résultats iront ici :\n{PATH_Result}")

Les résultats iront ici :
/mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Results


Enregistrement des données traitées au format "**parquet**" :

In [21]:
features_df.write.mode("overwrite").parquet(PATH_Result)

2024-01-29 13:07:49.355019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 13:07:49.355075: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 13:07:49.355095: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 13:07:49.360003: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 13:07:50.695910: I tensorflow/compiler/

## **Chargement des données enregistrées et validation du résultat**

On charge les données fraichement enregistrées dans un **DataFrame Pandas** :

In [22]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

In [23]:
df.head()

,path,label,features
0,file:/mnt/c/Users/Data Science/Desktop/OpenCla...,Apple Crimson Snow,"[0.0, 0.0, 0.014113676, 0.0, 0.0, 0.03997269, ..."
1,file:/mnt/c/Users/Data Science/Desktop/OpenCla...,Apple Crimson Snow,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.247497, 0.1838009,..."
2,file:/mnt/c/Users/Data Science/Desktop/OpenCla...,Apple Braeburn,"[0.6390394, 0.03641435, 0.0, 0.0, 0.0, 0.86568..."
3,file:/mnt/c/Users/Data Science/Desktop/OpenCla...,Apple Braeburn,"[0.8058856, 0.07883457, 0.0, 0.0, 0.0, 0.63891..."
4,file:/mnt/c/Users/Data Science/Desktop/OpenCla...,Apple Crimson Snow,"[0.015286029, 0.07301358, 0.0, 0.0, 0.0, 0.457..."


On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280 :

In [24]:
df.loc[0,'features'].shape

(1280,)